In [5]:
# 1. Import and install requirements
from logger import logger
from util.database import Database
from util.scraper import Scraper
import json
import itertools
import chromedriver_autoinstaller

database = Database("sqlite:///data/pdga_data.db")
scraper = Scraper()

chromedriver_autoinstaller.install()

[31/Oct/2024 13:24:31] INFO - Chromedriver is already installed.


'/Users/tobiasbeidlershenk/dev/repos/pdga-rating-bot/venv/lib/python3.9/site-packages/chromedriver_autoinstaller/130/chromedriver'

In [ ]:
# 2. Build list of courses and write to JSON
courses = scraper.get_courses_from_dgscene()
course_names = {
    course: scraper.get_readable_course_name(course) for course in courses
}
with open('data/course_names.json', 'w') as f:
    json.dump(course_names, f, indent=4)

In [ ]:
# 3. Build list of events and write to JSON
with open('data/course_names.json') as f:
    course_names: dict = json.load(f)

with open('data/course_events.json') as f:
    course_events: dict = json.load(f)

for i, course in enumerate(course_names):
    if course in course_events:
        logger.info(f'Skipping {course} (already scraped)...')
        continue

    logger.info(f'Fetching event {i}/{len(course_names)}')
    course_events[course] = scraper.get_all_sanctioned_events(course)
    logger.info(course_events[course])

    # periodically save ratings to file
    with open('data/course_events.json', 'w') as f:
        json.dump(course_events, f, indent=4)

logger.info("Done")

In [ ]:
# 4. Fetch ratings for each event and load into DB
try:
    logger.info('Fetching ratings...')
    
    with open('data/course_names.json') as f:
        course_names: dict = json.load(f)
    with open('data/course_events.json') as f:
        course_events: dict = json.load(f)
    
    for i, course in enumerate(course_events):
        events = course_events[course]
        rounds = []

        for j, event in enumerate(events):
            event_id = event['event_id']
            if database.event_exists(event_id):
                logger.info(f'Skipping {event_id} (already scraped)...')
                continue

            course_ratings = scraper.get_round_ratings_for_tournament(event_id)
            rounds.extend(course_ratings)
            logger.info(f'Event {j+1}/{len(events)} - Course {i+1}/{len(course_events)}')

        data = {
            'course_name': course,
            'readable_course_name': course_names[course],
            'events': course_events[course],
            'rounds': rounds
        }
        database.insert_course_data(data)
        

except BaseException as e:
    logger.info(f'Error fetching ratings: {e.with_traceback()}')
except KeyboardInterrupt as e:
    logger.info(f'Error fetching ratings: {e.with_traceback()}')

scraper.cleanup()
logger.info("Done")

In [24]:
from decimal import Decimal
import sys
from fuzzywuzzy import fuzz, process
import numpy as np
from collections import Counter

def get_ratings(course_name: str, layout_name: str, score: int) -> dict:
    rounds = database.query_all_course_rounds(course_name)
    if len(rounds) == 0:
        all_course_names = [course.readable_course_name for course in database.query_all_courses()]
        scored_course_names: tuple[str, int] = process.extractBests(course_name, all_course_names, scorer=fuzz.partial_ratio, score_cutoff=0, limit=5)
        similar_course_names = [course for course, _ in scored_course_names]
        sys.exit(f"No courses found for course: '{course_name}'. \nDid you mean: {', '.join(similar_course_names)}")

    all_layout_names = set([round.layout_name for round in rounds])
    scored_layouts: tuple[str, int] = process.extractBests(layout_name, all_layout_names, scorer=fuzz.partial_token_sort_ratio, score_cutoff=0, limit=10)
    best_layout_score = scored_layouts[0][1]
    if best_layout_score < 75:
        similar_layout_names = [layout for layout, _ in scored_layouts]
        sys.exit(f"No rounds found for layout: '{layout_name}'. \nDid you mean: {', '.join(similar_layout_names)}")

    if layout_name.lower() in [x.lower() for x in all_layout_names]:
        scored_layout_names = [layout_name]
    else:
        scored_layout_names = [layout for layout, _ in process.extractBests(layout_name, all_layout_names, scorer=fuzz.partial_token_sort_ratio, score_cutoff=75, limit=100)]
    
    scored_rounds = [round for round in rounds if round.layout_name in scored_layout_names]
    scored_layout_par_ratings = [round.par_rating for round in scored_rounds]
    scored_layout_stroke_values = [round.stroke_value for round in scored_rounds]
    matching_rounds = [(round.layout_name, round.layout_total_distance, round.layout_hole_distances, round.layout_par) for round in scored_rounds]
    matching_rounds_counter = Counter(matching_rounds)
    condensed_matching_rounds = [(w, x, y, z, count) for (w, x, y, z), count in matching_rounds_counter.items()]
    layout_distance_mode = np.bincount([round.layout_total_distance for round in scored_rounds]).argmax()
    par_rating_mean = np.mean(scored_layout_par_ratings)
    par_rating_median = np.median(scored_layout_par_ratings)
    par_rating_std = np.std(scored_layout_par_ratings)
    stroke_value_mean = np.mean(scored_layout_stroke_values)
    stroke_value_median = np.mean(scored_layout_stroke_values)
    par_rating_range = np.ptp(scored_layout_par_ratings)
    data = {
        'course_name': course_name,
        'layout_name': layout_name,
        'used_layouts': condensed_matching_rounds,
        'score': score,
        'layout_distance_chosen': int(layout_distance_mode),
        'par_rating_mean': float(par_rating_mean),
        'par_rating_median': float(par_rating_median),
        'par_rating_std': float(par_rating_std),
        'stroke_value_mean': float(stroke_value_mean),
        'stroke_value_median': float(stroke_value_median),
        'par_rating_range': int(par_rating_range)
    }
    return data

In [25]:
course_name = "Aperture Park"
layout_name = "Short Tees (whites)"
score = 7

rounds = database.query_all_course_rounds(course_name)
# print(set([round.layout_name for round in rounds]))

data = get_ratings(course_name, layout_name, score)
print(json.dumps(data, indent=4))
print(data['par_rating_mean'] - (score * data['stroke_value_median']))

{
    "course_name": "Aperture Park",
    "layout_name": "Short Tees (whites)",
    "used_layouts": [
        [
            "Short Tees (whites)",
            4004,
            "234, 312, 248, 275, 232, 181, 403, 226, 155, 213, 222, 185, 165, 209, 185, 240, 117, 202",
            56,
            4
        ]
    ],
    "score": 0,
    "layout_distance_chosen": 4004,
    "par_rating_mean": 799.5,
    "par_rating_median": 799.5,
    "par_rating_std": 0.5,
    "stroke_value_mean": 14.2175,
    "stroke_value_median": 14.2175,
    "par_rating_range": 1
}
799.5
